In [1]:
import json
import pandas as pd
import numpy as np
from copy import deepcopy
from IPython.display import display
import re

from modules.team import Team, BenchTeam
from modules.player import Player, Position
from modules.transfer import Transfer
from modules.fixture_difficulty_matrix import FixtureDifficultyMatrix
from modules.utils import getDataFilesSorted

import config

In [2]:
CURRENT_DATE = config.CURRENT_DATE
teams_filename = f"./results/{CURRENT_DATE}/results_{CURRENT_DATE}.json"
scores_filename = f"./results/{CURRENT_DATE}/scores_{CURRENT_DATE}.json"
SELECTED_MODEL_INDEX = 3
TOTAL_BUDGET = 1000

In [3]:
BB_THRESHOLD = 0.35

In [4]:
current_team_names = {"\u0110or\u0111e Petrovi\u0107",
                      "Alisson Becker",
                      "Jurriën Timber",
                      "Matty Cash",
                      "Marcos Senesi Bar\u00f3n",
                      "Reece James",
                      "Micky van de Ven",
                      "Yankuba Minteh",
                      "Jaidon Anthony",
                      "Idrissa Gana Gueye",
                      "Bruno Borges Fernandes",
                      "Carlos Henrique Casimiro",
                      "Jean-Philippe Mateta",
                      "Erling Haaland",
                      "Jarrod Bowen",
                      }

In [5]:
# TODO: Change how files are loaded

with open(teams_filename, "r") as f:
    tempJson: list[dict] = json.load(f)

actualJson = tempJson[SELECTED_MODEL_INDEX]

In [6]:
all_player_data = pd.DataFrame.from_records(actualJson["players"])

In [7]:
print(all_player_data)

     clean_sheets  cost  form  gameweek  ict_index   id  \
0             8.0    59   3.0        15       21.2    1   
1             0.0    42   0.0        15        0.0    2   
2             0.0    40   0.0        15        0.0    3   
3             0.0    39   0.0        15        0.0    4   
4             7.0    63   0.2        15       46.9    5   
..            ...   ...   ...       ...        ...  ...   
753           0.0    45   0.0        15        0.0  754   
754           0.0    45   0.0        15        0.0  755   
755           0.0    45   0.0        15        0.0  756   
756           0.0    40   0.0        15        0.0  757   
757           0.0    45   1.0        15        0.2  758   

                             name opposing_team  play_percent  \
0               David Raya Martín           AVL      0.933333   
1      Kepa Arrizabalaga Revuelta           AVL      0.000000   
2                       Karl Hein           AVL      0.000000   
3                   Tommy Setfo

In [8]:
currentTeamPlayers = all_player_data.loc[all_player_data["name"].isin(current_team_names)]


for player in current_team_names:
    if(player not in currentTeamPlayers["name"].values):
        raise ValueError(f"player '{player}' not found")

In [9]:
current_team = Team.fromDataFrame(currentTeamPlayers)

In [10]:
current_team_cost = current_team.getTotalCost()
current_team_cost

985

In [11]:
with open(teams_filename,"r",encoding="utf-8") as f:
    all_data = json.load(f)

all_data = all_data[SELECTED_MODEL_INDEX]
selected_team_df = pd.DataFrame.from_records(all_data["team"])

In [12]:
selected_team = Team.fromDataFrame(selected_team_df)

In [13]:
display(selected_team)

In [14]:
new_players = selected_team - current_team
print(new_players)


Total Score: 829.041

Goalkeepers:
- Robin Roefs	Score: 62.3	Cost: 47	Fixture Difficulty: 0.0
- Dean Henderson	Score: 61.27	Cost: 51	Fixture Difficulty: 0.0
Defenders:
- Tyrick Mitchell	Score: 68.94	Cost: 50	Fixture Difficulty: 0.0
- Maxence Lacroix	Score: 73.89	Cost: 51	Fixture Difficulty: 0.0
- Daniel Muñoz Mejía (Captain) 	Score: 80.87	Cost: 61	Fixture Difficulty: 0.0
- Riccardo Calafiori	Score: 68.93	Cost: 58	Fixture Difficulty: 0.0
Attackers:
- João Pedro Junqueira de Jesus	Score: 68.66	Cost: 74	Fixture Difficulty: 0.0
- Igor Thiago Nascimento Rodrigues (Vice Captain) 	Score: 78.88	Cost: 68	Fixture Difficulty: 0.0
Midfielders:
- Bryan Mbeumo	Score: 63.95	Cost: 86	Fixture Difficulty: 0.0
- Pedro Lomba Neto	Score: 65.31	Cost: 72	Fixture Difficulty: 0.0
- Declan Rice	Score: 72.87	Cost: 71	Fixture Difficulty: 0.0
- Enzo Fernández	Score: 63.16	Cost: 67	Fixture Difficulty: 0.0


In [15]:
new_team_cost = selected_team.getTotalCost()
new_team_cost

added_cost = new_team_cost - current_team_cost
print("Added cost:",added_cost)

surplus = current_team_cost - new_team_cost


Added cost: 0


In [16]:
#HEURISTIC = "combined"
#MODE = SolverMode.CHEAPEST_FIRST

In [17]:
# all_player_data["score"] = all_player_data[HEURISTIC] * all_player_data["form"]
deviation = np.std(all_player_data["score"])
scale_factor = deviation
scale_factor, deviation

(20.232327448252416, 20.232327448252416)

In [18]:
matrix = FixtureDifficultyMatrix()
matrix.precomputeFixtureDifficulty(0, config.CURRENT_GAMEWEEK+1, 3, config.CURRENT_SEASON, 1.0)

new_players.recalculateFixtureDifficulty(matrix)
current_team.recalculateFixtureDifficulty(matrix)
# new_players.calculateScore(HEURISTIC)

# selected_team.recalculateFixtureDifficulty(matrix)
# selected_team.calculateScore(HEURISTIC)

# current_team.recalculateFixtureDifficulty(matrix)
# current_team.calculateScore(HEURISTIC)

In [19]:
print("Current Team:")
display(current_team)
print()
print("Selected Team:")
display(selected_team)

Current Team:



Selected Team:


In [20]:
score_dif = selected_team.getTotalScore() - current_team.getTotalScore()
print("Score difference:",score_dif)

Score difference: 168.73875000000004


In [21]:
def getBest(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team, pPosition: Position):
    # TODO: Store Team players in sorted tree instead of list
    currentPositionData = pCurrentTeam.getPlayersListByPosition(pPosition)
    newPlayersPositionData = pNewPlayers.getPlayersListByPosition(pPosition)
    oldTotalCost = pCurrentTeam.getTotalCost()
    maxCost = max(oldTotalCost, TOTAL_BUDGET)
    #maxCost = TOTAL_BUDGET

    allTransfers = []

    for i in range(len(currentPositionData)):
        oldPlayer = currentPositionData[i]
        oldPlayerCost = oldPlayer.getCost()
        for j in range(len(newPlayersPositionData)):

            newPlayer = newPlayersPositionData[j]
            newCost = oldTotalCost - oldPlayerCost + newPlayer.getCost()

            if (newCost <= maxCost):
                allTransfers.append(Transfer(oldPlayer, newPlayer))

    if (len(allTransfers) >= 1):
        bestTransfer = max(allTransfers)
        if(bestTransfer.getScoreDif() > 0):
            return bestTransfer
    else:
        return None

In [22]:
def get_updated_team(team: pd.DataFrame, old_player: pd.Series, new_player: pd.Series):
    team: pd.DataFrame = team.drop(index=old_player.name)
    team.loc[len(team)] = new_player
    return team

In [23]:
def get_bench(team: pd.DataFrame):
    positions = ["FWD","DEF","MID","GKP"]
    team = team.reset_index()
    bench = pd.DataFrame(columns=team.columns)
    for position in positions:
        worst_player_index = team.loc[team["position"]==position]["score"].idxmin()
        worst_player = team.loc[worst_player_index].copy()
        bench.loc[len(bench)] = worst_player
        team = team.drop(index=worst_player_index)
    return team, bench

In [24]:
def getBestTransferNew(pCurrentTeam: Team, pNewTeam: Team, pNewPlayers: Team) -> Transfer | None:
    positions = Position.listValues()
    bestTransfers = []
    for position in positions:
        transfer = getBest(pCurrentTeam, pNewTeam, pNewPlayers, position)
        if(transfer is not None):
            bestTransfers.append(transfer)
    actualBestTransfer = max(bestTransfers)
    return actualBestTransfer

In [25]:
def getNewTeam(pCurrentTeam: Team, pSelectedTeam: Team, pNewPlayers: Team, pMatrix: FixtureDifficultyMatrix):

    TRIPLE_CAPT_THRESHOLD = 0.1

    transferData = getBestTransferNew(pCurrentTeam, pSelectedTeam, pNewPlayers)
    transferData.updateFixtureDifficulties(pMatrix)
    print("Best transfer:")
    print(transferData)
    oldPlayer = transferData.getOldPlayer()
    newPlayer = transferData.getNewPlayer()
    if transferData is None:
        return
    
    changingPosition: Position = transferData.getPosition()
    
    newTeam = deepcopy(pCurrentTeam)
    playersOfPosition = newTeam.getPlayersListByPosition(changingPosition)

    for i in range(len(playersOfPosition)):
        player = playersOfPosition[i]
        if (player.getId() == oldPlayer.getId()):
            newTeam.removePlayerByIndex(i, changingPosition)
            newTeam.addPlayer(newPlayer)
    

    newTeam.recalculateFixtureDifficulty(pMatrix)

    for player in newTeam.getPlayers():
        if(player.getCurrentDifficulty() < TRIPLE_CAPT_THRESHOLD):
            print(f"Suggested triple captain: {player.getName()}")
    return newTeam

In [26]:
def calcDecay() -> float:
    halfNumGameweeks: int = config.MAX_GAMEWEEKS // 2
    remainingWeeks = halfNumGameweeks - config.CURRENT_GAMEWEEK
    percentLeft: float = remainingWeeks / halfNumGameweeks
    actualThreshold = ((1.0 - percentLeft) * config.MAX_CHIP_DECAY)
    return actualThreshold
    ...

In [27]:
def averageDifficulty(pPlayers: list[Player]) -> float:
    totalFixtureDifficulty: float = 0.0
    for player in pPlayers:
        totalFixtureDifficulty += player.getCurrentDifficulty()
    avgDifficulty: float = totalFixtureDifficulty / len(pPlayers)
    return avgDifficulty

In [28]:
def checkBenchBoost(pTeam: BenchTeam):

    avgDifficulty: float = averageDifficulty(pTeam.getBenchPlayerList())
    
    actualThreshold = calcDecay() + BB_THRESHOLD

    if (avgDifficulty <= actualThreshold):
        toPrint = "Bench boost is suggested. Threshold: {:.3f} Average Fixture Difficulty: {:.3f}".format(actualThreshold, avgDifficulty)
        display(toPrint)
    else:
        toPrint = "Bench boost NOT needed. Threshold: {:.3f} Average Fixture Difficulty: {:.3f}".format(actualThreshold, avgDifficulty)
        display(toPrint)

In [29]:
def checkFreeHit(pOldTeam: BenchTeam, pNewTeam: BenchTeam):
    oldTeamPlayers = set(x.getName() for x in pOldTeam.getAllPlayerList())
    newTeamPlayers = set(x.getName() for x in pNewTeam.getAllPlayerList())
    uniqueNewPlayers = newTeamPlayers - oldTeamPlayers
    diffPercent = len(uniqueNewPlayers) / len(newTeamPlayers)
    
    THRESHOLD = 0.9
    actualThreshold = THRESHOLD - calcDecay()

    avgDifficulty: float = averageDifficulty(pNewTeam.getAllPlayerList())
    FREE_HIT_DIFF_THRESHOLD = 0.45
    stats = "Threshold: {:.3f} Team difference: {:.3f} Difficulty: {:.3f}".format(actualThreshold, diffPercent, avgDifficulty)

    if ((diffPercent >= actualThreshold) and (avgDifficulty <= FREE_HIT_DIFF_THRESHOLD)):
        display(f"Free Hit recommended. {stats}")
    else:
        display(f"Free Hit NOT needed. {stats}")

In [30]:
newTeam = getNewTeam(current_team, selected_team, new_players, matrix)
newTeam = newTeam.toBenchTeam()
display(newTeam)
display(f"New total score: {newTeam.getTotalScore()}")
display(f"New total cost: {newTeam.getTotalCost()}")
checkBenchBoost(newTeam)
checkFreeHit(newTeam, selected_team.toBenchTeam())

Best transfer:
Transfer from Reece James -> Daniel Muñoz Mejía:
Old player: Reece James	Score: 15.12	Cost: 56	Fixture Difficulty: 0.6666666666666666
New player: Daniel Muñoz Mejía (Captain) 	Score: 53.91	Cost: 61	Fixture Difficulty: 0.33333333333333337
- Cost change: 5
- Score change: 38.79


'New total score: 536.0295833333333'

'New total cost: 990'

'Bench boost is suggested. Threshold: 0.597 Average Fixture Difficulty: 0.506'

'Free Hit recommended. Threshold: 0.653 Team difference: 0.733 Difficulty: 0.393'